In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Георгий\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
with open("intents.json") as file:
    data = json.load(file)

In [4]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern) #разбивает строку поэлементно
        words.extend(wrds) #разбивает элементы по буквам и добавляет в массив
        docs_x.append(wrds) #в массив добавляет массивы по паттернам
        docs_y.append(intent["tag"]) #добавляет название pattern
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"]) #добавляет название pattern (только уникальные)
               
words = [stemmer.stem(w.lower()) for w in words if w != "?"] #делаем все слова маленькими буквами и убираем знаки вопроса
words = sorted(list(set(words))) #для лучшей модели обучения
labels = sorted(labels) #и сортируем words & labels

training = []
output = []

empty_out = [0 for _ in range(len(labels))] #создаем пустой список, длины списка labels

for x, d in enumerate(docs_x): #нумеруем все вопросы
    bag = []
    wrds = [stemmer.stem(w) for w in d]
    for w in words:
        if w in wrds:
            bag.append(1) #узнаем сходство вопроса и предполагаемого ответа по нему и добавляем в bag, а дальше в training
        else:
            bag.append(0)
    output_row = empty_out[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)
    
training = numpy.array(training)
output = numpy.array(output)

In [5]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
try:
    model.load("model.tflearn")
except:
    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from C:\Users\Георгий\Desktop\model.tflearn


In [7]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1       
    return numpy.array(bag)

In [8]:
def chat():
    print("Start Talking with the bot(type quit to stop!)")
    while True:
        inp = input("You: ")
        if inp.lower() == 'quit':
            break
            
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.5:
            for tg in  data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
            print("\n")
            
            
        else:
            
            print("I didnt get that, try again")

In [ ]:
chat()

Start Talking with the bot(type quit to stop!)
You: cut
Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.


